In [1]:
import os
import time
import torch
import random
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from exp.exp_main import Exp_Main
from data_provider.data_factory import data_provider
from torch.utils.data import DataLoader
from utils.timefeatures import time_features
from utils.tools import EarlyStopping, adjust_learning_rate, visual
from layers.Embed import DataEmbedding, DataEmbedding_wo_pos
from layers.AutoCorrelation import AutoCorrelation, AutoCorrelationLayer
from layers.FourierCorrelation import FourierBlock, FourierCrossAttention
from layers.MultiWaveletCorrelation import MultiWaveletCross, MultiWaveletTransform
from layers.SelfAttention_Family import FullAttention, ProbAttention
from layers.Autoformer_EncDec import Encoder, Decoder, EncoderLayer, DecoderLayer, my_Layernorm, series_decomp, series_decomp_multi
import math
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

%matplotlib inline





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



fix_seed = int((time.time() * 1000) % 2**32)
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


class Configs:
    # Basic configuration
    is_training = 1  # status, default 1
    task_id = 'train'  # task id, default 'test'
    model = 'FEDformer'  # model name, options: [FEDformer, Autoformer, Informer, Transformer], default 'FEDformer'

    # Supplementary configuration for FEDformer model
    version = 'Fourier'  # options: [Fourier, Wavelets], default 'Fourier'
    mode_select = 'random'  # mode selection method, options: [random, low], default 'random'
    modes = 64  # modes to be selected randomly, default 64
    L = 3  # ignore level, default 3
    base = 'legendre'  # mwt base, default 'legendre'
    cross_activation = 'tanh'  # mwt cross attention activation function, options: [tanh, softmax], default 'tanh'

    # Data loader configuration
    data = 'custom'  # dataset type, default 'ETTh1' set 'custom' for own data
    root_path = ''  # root path of the data file, default './dataset/ETT/'
    data_path = 'tr_EURUSD=X_1h_14.csv'  # data file, default 'ETTh1.csv'
    features = 'MS'  # forecasting task, options: [M, S, MS], default 'M'
    target = 'Close'  # target feature in S or MS task, default 'OT'
    freq = 't'  # frequency for time features encoding, options: [s, t, h, d, b, w, m], default 'h'
    detail_freq = 't'  # detailed frequency, default help='like freq, but use in predict'
    checkpoints = './checkpoints/'  # location of model checkpoints, default './checkpoints/'

    # Forecasting task configuration
    seq_len = 36  # input sequence length, default 96
    label_len = 18  # start token length, default 48
    pred_len = 36  # prediction sequence length, default 96

    # Model definition
    enc_in = 13  # encoder input size, default 7
    dec_in = 13  # decoder input size, default 7
    
    c_out = 1  # output size, default 7
    d_model = 512  # dimension of model, default 512 i was getting nan if itrained my model on 51
    n_heads = 8  # number of heads, default 8
    e_layers = 2  # number of encoder layers, default 2
    d_layers = 1  # number of decoder layers, default 1
    d_ff = 2048  # dimension of fcn, default 2048
    
    moving_avg = 6  # window size of moving average, default [24]
    
    factor = 1  # attention factor, default 1
    distil = True  # whether to use distilling in encoder, default True
    
    dropout = 0.09  # dropout, default 0.05 --
    
    embed = 'timeF'  # time features encoding, options: [timeF, fixed, learned], default 'timeF' if timeF no time encoding
    activation = 'gelu'  # activation, default 'gelu'
    output_attention = False  # whether to output attention in encoder, default False
    do_predict = False  # whether to predict unseen future data, default False

    # Optimization configuration
    num_workers = 2  # data loader number of workers, default 10s
    # itr = 3               # experiments times, default 3 this is not equal to no of epochs
    train_epochs = 10 # train epochs, default 10
    batch_size = 32        # batch size of train input data, default 32
    patience = 3            # early stopping patience, default 3
    learning_rate = 0.0001  # optimizer learning rate, default 0.0001
    # des = 'Exp'           # experiment description, default 'test'
    loss = 'mse'            # loss funnction, default 'mse'
    lradj = 'type3'         # adjust learning rate, default 'type1'
    use_amp = False        # use automatic mixed precision training, default False

    # GPU configuration
    use_gpu = True  # use GPU, default True
    gpu = 0  # GPU, default 0
    use_multi_gpu = False  # use multiple GPUs, default False
    devices = '0,1'  # device ids of multiple GPUs, default '0,1'




settings = Configs()
Exp = Exp_Main



setting = '{}_{}_{}_modes{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}'.format(
    settings.task_id,
    settings.model,
    settings.mode_select,
    settings.modes,
    settings.data,
    settings.features,
    settings.seq_len,
    settings.label_len,
    settings.pred_len,
    settings.d_model,
    settings.n_heads,
    settings.e_layers,
    settings.d_layers,
    settings.d_ff,
    settings.factor,
    settings.embed,
    settings.distil,
    # settings.des,
    0)

exp = Exp(settings)  # set experiments
print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
model = exp.train(setting)
print('training finished')




Use GPU: cuda:0
fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
 fourier enhanced cross attention used!
modes_q=27, index_q=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
modes_kv=18, index_kv=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
enc_modes: 18, dec_modes: 27
>>>>>>>start training : train_FEDformer_random_modes64_custom_ftMS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_0>>>>>>>>>>>>>>>>>>>>>>>>>>
not scaling
train 662
not scaling
val 71
not scaling
test 174
running the training now
Epoch: 1 cost time: 14.105525732040405
Epoch: 1, Steps: 20 | Train Loss: 0.1671475 Vali Loss: 0.0300011 Test Loss: 0.0309815
Validation loss decreased (inf --> 0.030001).  Saving model ...
Updating learning rate to 0.0

KeyboardInterrupt: 

In [ ]:
model_parameters = sum(p.numel() for p in model.parameters())
print('total number of parameter in this model is {}'.format(model_parameters))
model_size_bytes = model_parameters * 4  # Assuming float32 weights
model_size_kb = model_size_bytes / 1024  # Convert bytes to kilobytes
print("Model size in KB:", model_size_kb)
print("Model size in MB:", model_size_kb//1024)


In [ ]:
from data_provider.data_loader import Dataset_Pred
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
timeenc = 0 if settings.embed != 'timeF' else 1


data_set = Dataset_Pred(settings.root_path,flag='pred',size=[settings.seq_len, settings.label_len, settings.pred_len],
                        features= settings.features, data_path='val_EURUSD=X_1h_14.csv', target=settings.target, scale=False, 
                        inverse=False, timeenc=1, freq=settings.freq, cols=None)




batch_x, batch_y, batch_x_mark, batch_y_mark = data_set.__getitem__(0)




batch_x = torch.tensor(batch_x)
batch_x = batch_x.unsqueeze(0)
batch_y = torch.tensor(batch_y)
batch_y = batch_y.unsqueeze(0)
batch_x_mark = torch.tensor(batch_x_mark)
batch_x_mark = batch_x_mark.unsqueeze(0)
batch_y_mark = torch.tensor(batch_y_mark)
batch_y_mark = batch_y_mark.unsqueeze(0)







print('shapes:')
print('batch_x',batch_x.shape)
    # print('batch_x',batch_x.round(7))

print('batch_x_mark.shape',batch_x_mark.shape)
print('batch_y',batch_y.shape)
print('batch_y_mark.shape',batch_y_mark.shape)




batch_x = batch_x.float().to(device)
batch_y = batch_y.float().to(device)
batch_x_mark = batch_x_mark.float().to(device)
batch_y_mark = batch_y_mark.float().to(device)

    #     # decoder input
    # dec_inp = torch.zeros_like(batch_y[:, -settings.pred_len:, :]).float()
    # dec_inp = torch.cat([batch_y[:, :settings.label_len, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder
    # outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
outputs = model.forward(batch_x, batch_x_mark, batch_y, batch_y_mark)


    # prep output to be furter detransfrormed and printed as chart
outputs = outputs.detach().cpu().numpy().squeeze()
outputs = outputs.reshape(-1, 1)

batch_y = batch_y.detach().cpu().numpy().squeeze()
batch_y = batch_y.reshape(-1, 1)

batch_x = batch_x.detach().cpu().numpy().squeeze()
batch_x = batch_x.reshape(-1, 1)




#plotting

# plt.plot(outputs , color='g')
# plt.plot(batch_x,color='b')




print(outputs.shape)

In [ ]:

import yfinance as yf
import pandas as pd



df_val = pd.read_csv('val_EURUSD=X_1h_14.csv') 
df_te = pd.read_csv('te_EURUSD=X_1h_14.csv')

y_true = df_te[['Close']].values
x_val = df_val[['Close']].values

# print(x_val)


In [ ]:
import matplotlib.pyplot as plt
import torch.nn as nn

import numpy as np
dates = pd.date_range('20230101', periods=36)  # Dummy date range
test_data = x_val.flatten()
true_data = y_true.flatten()
concatenated_array = np.concatenate((test_data, true_data))
# print(concatenated_array)

predicted_data = outputs
print(predicted_data.shape)
print(true_data.shape)


# Plotting
plt.figure(figsize=(10, 6))

plt.plot( concatenated_array, label='Test Data', color='blue')
# plt.plot(test_data, label='True Data', color='green')

# Adding title and labels
plt.title('Comparison of Test, True, and Predicted Data')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

# Show plot
plt.show()
plt.plot(predicted_data, label='Predicted Data', color='red')
